<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Préparation des données</a></span></li><li><span><a href="#Satisfaction-de-contraintes" data-toc-modified-id="Satisfaction-de-contraintes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Satisfaction de contraintes</a></span><ul class="toc-item"><li><span><a href="#Par-défintion" data-toc-modified-id="Par-défintion-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Par défintion</a></span></li></ul></li><li><span><a href="#Machine-Learning" data-toc-modified-id="Machine-Learning-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Machine Learning</a></span><ul class="toc-item"><li><span><a href="#KNN" data-toc-modified-id="KNN-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>KNN</a></span></li><li><span><a href="#Random-Forrest" data-toc-modified-id="Random-Forrest-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Random Forrest</a></span></li><li><span><a href="#MLP" data-toc-modified-id="MLP-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>MLP</a></span></li></ul></li></ul></div>

In [1]:
# IMPORT
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import sklearn
import random as rand
import time
import networkx as nx
import json


from sklearn import datasets
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

In [2]:

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

def precision(test, obtained, positive, negative):
    u = len(test)
    TP = 0
    FP = 0
    for i in range(u):
        if obtained[i] == positive and test[i] == positive :
            TP = TP +1
        elif obtained[i] == positive and test[i] == negative :
            FP = FP +1
    out = TP / (TP + FP +0.001) 
    return out


def recall(test, obtained, positive, negative):
    u = len(test)
    TP = 0
    FN = 0
    for i in range(u):
        if obtained[i] == positive and test[i] == positive :
            TP = TP +1
        elif obtained[i] == negative and test[i] == positive :
            FN = FN +1
    out = TP / (TP + FN +0.001) 
    return out


def f1_score(test, obtained, positive, negative):
    pre = precision(test, obtained, positive, negative)
    rec = recall(test, obtained, positive, negative)
    out = 2 * (pre * rec)/(pre + rec +0.001)
    return out

# Préparation des données

In [9]:
    

Data_Autism = pd.read_csv('data.csv', on_bad_lines='skip') 


np_Data_Autism = Data_Autism.to_numpy()
u, v = np_Data_Autism.shape
num_Data_Autism = np.zeros((u,v))  

#We are doing this by column
for j in range(v) :
    line = pd.unique(np_Data_Autism[:, j ])
    size = len(line)
    for i in range(u) :
        if isinstance(np_Data_Autism[i,j], int) or isinstance(np_Data_Autism[i,j], float) :
            if math.isnan(np_Data_Autism[i,j]) :
                num_Data_Autism[i,j] = 1000 #nan values, we need int to use the functions
            else :
                num_Data_Autism[i,j] = np_Data_Autism[i,j]
        elif np_Data_Autism[i,j].isdigit() : 
            num_Data_Autism[i,j] = np_Data_Autism[i,j]    
        else :
            # only non-number value need to be parametrise
            # we replace by (-1) * the place in np.unique()
            for index in range(size):
                if np_Data_Autism[i,j] == line[index] :
                    num_Data_Autism[i,j] =  index # modify st y is either 0 or 1
                    #standardisation : if it is a parametrise value, then it is negative

Data_Autism.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,gender,ethnicity,jundice,autsim,country_o_res,used_app_before,result,age_desc,relation,class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,'United States',no,6,'18 and more',Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5,'18 and more',Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8,'18 and more',Parent,YES
3,1,1,0,1,0,0,1,1,0,1,...,f,White-European,no,yes,'United States',no,6,'18 and more',Self,NO
4,1,0,0,0,0,0,0,1,0,0,...,f,?,no,no,Egypt,no,2,'18 and more',?,NO


In [6]:
np.random.shuffle(num_Data_Autism)

#Tout ce qui n'est pas y
x_R = num_Data_Autism[:,1:13]
x_L = num_Data_Autism[:,15:]
X = np.concatenate((x_R, x_L), axis=1)

# juste les données liées au comportement
X = num_Data_Autism[:,0:10]  

y = num_Data_Autism[:,14]

au_X_tr, au_X_te, au_y_tr, au_y_te = train_test_split( X, y, test_size=0.3, random_state=42)


# Satisfaction de contraintes

## Par défintion

In [16]:
def csp_def(dataX):
    u,v = dataX.shape
    dataY = np.zeros(u)
    
    for i in range(u):
        if sum(dataX[i,:]) > 6 :
            dataY[i] = 1
    return dataY


In [18]:
#print(csp_def(X))
csp_test_Y = csp_def(X)

u, v = X.shape
count = 0
for i in range(u):
    if csp_test_Y[i] == y[i]:
        count = count +1




pro = count/u
pre = precision(csp_test_Y, y, 1,0)
rec = recall(csp_test_Y, y,1,0)
f1_ = f1_score(csp_test_Y, y, 1,0)

print('There are ', count,' right guess out of ', u,' which is equal to ', pro*100, '% of right answers.')
print('precision  : ', pre,'\nrecall : ', rec, '\nF1 score : ',f1_)

There are  510  right guess out of  704  which is equal to  72.44318181818183 % of right answers.
precision  :  0.4725222799749453 
recall :  0.2275120237459061 
F1 score :  0.3067025382681907


# Machine Learning

## KNN

In [19]:
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(au_X_tr, au_y_tr)

KNeighborsClassifier(n_neighbors=7)

In [20]:
prd_au_y = neigh.predict(au_X_te)


u, v = au_X_te.shape
count = 0
for i in range(u):
    if prd_au_y[i] == au_y_te[i]:
        count = count +1




pro = count/u
pre = precision(au_y_te, prd_au_y, 1,0)
rec = recall(au_y_te, prd_au_y, 1,0)
f1_ = f1_score(au_y_te, prd_au_y, 1,0)


print('Il y a ', len(au_y_te),' training data et ', len(au_y_te), ' testing data.')
print('There are ', count,' right guess out of ', u,' which is equal to ', pro*100, '% of right answers.')
print('precision  : ', pre,'\nrecall : ', rec, '\nF1 score : ',f1_)

Il y a  212  training data et  212  testing data.
There are  183  right guess out of  212  which is equal to  86.32075471698113 % of right answers.
precision  :  0.9990009990009991 
recall :  0.033332222259258026 
F1 score :  0.0644495360273245


## Random Forrest

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(au_X_tr, au_y_tr)

RandomForestClassifier(max_depth=10, random_state=0)

In [22]:
prd_au_y = clf.predict(au_X_te)


u, v = au_X_te.shape
count = 0
for i in range(u):
    if prd_au_y[i] == au_y_te[i]:
        count = count +1




pro = count/u
pre = precision(au_y_te, prd_au_y, 1,0)
rec = recall(au_y_te, prd_au_y,1,0)
f1_ = f1_score(au_y_te, prd_au_y, 1,0)


print('Il y a ', len(au_y_te),' training data et ', len(au_y_te), ' testing data.')
print('There are ', count,' right guess out of ', u,' which is equal to ', pro*100, '% of right answers.')
print('precision  : ', pre,'\nrecall : ', rec, '\nF1 score : ',f1_)

Il y a  212  training data et  212  testing data.
There are  178  right guess out of  212  which is equal to  83.9622641509434 % of right answers.
precision  :  0.3333055578701775 
recall :  0.1333288890370321 
F1 score :  0.1900598214464592


## MLP

In [23]:

hid_l =[10, 10, 10]  # MODIFY THIS; DON T TOUCH THE REST

time_start = time.time()

clf = MLPClassifier(learning_rate_init= 0.0001, alpha=1e-05, hidden_layer_sizes=hid_l)
clf.fit(au_X_tr, au_y_tr)
MLPClassifier(learning_rate_init= 0.0001, alpha=1e-05, hidden_layer_sizes=hid_l)
prd_mush_y = clf.predict(au_X_te)

time_elapsed = (time.time() - time_start)

u, v = au_X_te.shape
count = 0
for i in range(u):
    if prd_au_y[i] == au_y_te[i]:
        count = count +1




pro = count/u
pre = precision(au_y_te, prd_au_y, 1,0)
rec = recall(au_y_te, prd_au_y,1,0)
f1_ = f1_score(au_y_te, prd_au_y,1,0)


print('Il y a ', len(au_y_te),' training data et ', len(au_y_te), ' testing data.')
print('There are ', count,' right guess out of ', u,' which is equal to ', pro*100, '% of right answers.')
print('precision  : ', pre,'\nrecall : ', rec, '\nF1 score : ',f1_)

Il y a  212  training data et  212  testing data.
There are  178  right guess out of  212  which is equal to  83.9622641509434 % of right answers.
precision  :  0.3333055578701775 
recall :  0.1333288890370321 
F1 score :  0.1900598214464592


/home/nes/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
